In [1]:
import cmath as cm
import numpy as np

# Conformal Mappings and Object-Oriented Programming
This notebook addresses how conformal mapping may be implemented in Python, first by using functions, then through Object classes, which are introduced and explained.

## Conformal mappings with functions
Conformal mappings may be conveniently encoded through a trick called a function closure. Basically, an outer function defines an "environment" or "scope," (programming terms which define where a particular variable is defined and able to be used) with some variables in it which are used in an inner function. The inner function is then returned and used in a different environment, but it remembers the variables from the environment where it was originally defined. See the code below for an illustration of how this works for the Schwartz-Christoffel Transformation (see Module section 3.5 at this writing)

In [31]:
from scipy.integrate import quad

def SCT(xPoints, wVertices, interiorAngles):
    """
    Given the array of x-locations on the real axis, the locations in the w-plane these points are
    mapped to, and the interior angles, measured in the w-plane, of the resultant polygon, this
    function returns another function which will map a given point or points in the z-plane to the 
    w-plane by the Schwartz-Christoffel Tranformation
    
    (This is an example of a docstring in a function, it is so other people can understand the code 
    and the meaning of the input)
    
    Inputs:
    -------
        xPoints: array-like 
            Represents x_1, x_2,...x_n-1 locations on real axis which are mapped to
            w_1, w_2,...w_n-1 in the complex plane for an n-sided polygon
        wVertices: array-like
            represents w_1, w_2,...w_n-1 vertices of the resultant polygon. Do not include
            w_n, because if the polygon is closed x=inf is mapped to w_n
        interiorAngles: array-like
            the interior angles theta_1, theta_2,...theta_n-1 of the resultant polygon in the w-plane
            
    Returns:
        transform: callable
            Function which can be called as w = tranform(z) to find the image of z under the
            particular Schwartz-Christoffel transformation.
    """
    
    # input checking
    w = np.array(wVertices, dtype=np.complex128)
    theta = np.array(interiorAngles, dtype=np.complex128)
    x = np.array(xPoints, dtype=np.complex128)
    Nm1 = len(x) # number of points minus 1
    assert len(theta) == Nm1 and len(w) == Nm1, "Input values must all be arrays of the same length"
    
    # SCT uses an indefinite integral (which requires analyticity of the
    # integrand, but that is satisfied generally speaking), to do this numerically 
    # we need some sort of definite (contour) integral. Can go from the origin to z on
    # a straight line (or maybe better to go along x-axis and then along arc of radius r)
    
    # here's the SCT integrand with numpy logic
    def integrand(z):
        return np.prod((z-x)**(theta/np.pi-1))
    
    # also, we need to evaluate the integral for the given w-vertices to solve for alpha and beta
    # before we can return the function
    
    # straight line contour integrals to the defined map points
    # x is real so no need to divide the integral
    integrals = np.zeros(x.shape)
    for i in np.arange(Nm1):
        integrals[i], _ = quad(lambda t: x[i]*integrand(x[i]*t), 0, 1)
        print(f'error of {i}th integral', _) # the error
    
    linEqnMatrix = np.column_stack((integrals, np.ones(x.shape))) # n-1 x 2 matrix of alpha, beta coefficients
    # the system is over-determined but should be consistent, so can use least squares or just first two rows
    [alpha, beta], resid, _, _ = np.linalg.lstsq(linEqnMatrix, w, rcond=None)
    
    print('alpha:', alpha, 'beta:', beta) # these are not right, supposed to be 1/pi and 0, instead approx. 0 and 1/2
    def transform(z):
        real, _ = quad(lambda t: (z*integrand(z*t)).real,0,1)
        print('real error:', _)
        imag, _ = quad(lambda t: (z*integrand(z*t)).imag,0,1)
        print('imag error:', _)
        return alpha * (real+imag*1j) + beta
    
    return transform

# test code (compare to Module section 3.5)
x = [-1, 1]
w = [1j, 0]
theta = [3*np.pi/2, np.pi/2]
step = SCT(x, w, theta)
print(step(-2.1+0.1j)) # should be something with +1j for the imaginary part. ## some sort of logic error ## or maybe quadrature error
print(alpha, beta) # produces an error -- NameError

error of 0th integral 1.750622044263571e-19
error of 1th integral 5.970694498641696e-17
alpha: 3.0118989862952563e-17j beta: 0.4999999999999998j
real error: 3.1501324661291964e-10
imag error: 1.4683529196502502e-09
(-1.8986768463939602e-17+0.4999999999999998j)


NameError: name 'alpha' is not defined

In the example above, the constants alpha and beta are computed in the body of SCT and used within the `transform` function. They cannot be accessed outside the function `SCT`, as illustrated by the last line of code producing an error. However, the function `step` returned by `SCT` is clearly able to access their values and compute the correct answer.

## Conformal mapping with objects
code like the SCT above is very handy, but what if you wanted to know alpha and beta after they were computed? What if you wanted to easily double check that the set of angles fed to it was correct? What if you wanted to evaluate the inverse transform easily, since it would use the same set of parameters but could require an entirely different function?

Objects provide the answer to these types of questions by combining several relevant pieces of data or 'fields' and functions or 'methods' under one interface. We have already met a few objects, such as Python's `complex` data type, which has the fields `.real` and `.imag` and the method `.conjugate`, as well as numpy arrays, matplotlib figures and axes, and really everything in Python. Here though, we will look at how such objects are made and how we can use that interface for defining conformal mappings.

Below is an example of how a simple transformation like a rotation may be encoded and used as a 'class,' a programming construct which defines an object.

In [8]:
class Rotation():
    
    # first, the method that allows the creation of an object,
    # __init__() is not typically called by the user of a class
    # rather, the Python Interpreter invokes it when a user writes
    # Rotation(theta)
    def __init__(self, theta):
        self._theta = theta
        # self is conventionally used to represent the object within the class
        # but any word can be used (Java programmers are free to substitute `this`
        # for example)
        
        # We do not want the user easily accessing theta, because
        # once the rotation is created, we want it to stay the same,
        # thus the _theta (though the user is not prevented from accessing it, 
        # the name is less obvious)
    
    # we do want the user to be able to see what the value of theta is,
    # so this method, a 'getter,' provides that capability. If theta was to be allowed to change
    # we would uncomment the method below this one.
    @property
    def theta(self):
        return self._theta
    
#     @property.setter
#     def theta(self, new_theta):
#         self._theta = new_theta
    
    # this method will return the result of an inverse rotation
    # through the same angle
    def inv(self, z):
        return cm.exp(-self._theta*1j)*z
    
    # like __init__, this is a special method the Python Interpreter
    # automatically looks for when a user invokes a function. so if 
    # r is a Rotation by 90 degrees, ie, r = Rotation(np.pi/2) has been invoked
    # then r(1) will return 1j by calling r.__call__(1)
    def __call__(self, z):
        return cm.exp(self._theta*1j)*z
    
    # an example of a factory method, gives an alternate way
    # to instantiate the class/create an object because it calls the
    # constructor.
    @classmethod
    def inDegrees(cls, theta_deg): # passes the class instead of the object
        return cls(theta_deg/180*np.pi)
    
# test code:
r = Rotation(np.pi/4)
z = 2
print("Rotation of {} by {} radians yields {}".format(z, r.theta, r(z)))

r2 = Rotation.inDegrees(90) # use of factory method
print(r2(z)) # approx 2j is expected
print(r2.inv(r2(z)) == z) # True is expected

# this illustrates accessibility of r._theta vs. r.theta
print(r._theta)
r._theta = np.pi/2 # sneakily set theta with the 'private' variable
print(r.theta==np.pi/2) # call the getter to reveal the change.
# trying to set to the getter results in an error, and so theta's value is usually protected
r.theta = np.pi

Rotation of 2 by 0.7853981633974483 radians yields (1.4142135623730951+1.4142135623730951j)
(1.2246467991473532e-16+2j)
True
0.7853981633974483
True


AttributeError: can't set attribute

## Exercises

1. Write a class for Mobius Tranforms using the structure below as a guide.

In [ ]:
class Mobius():
    """
    write a good doc string
    
    """
    
    def __init__(self, alpha, beta, gamma, delta): # make the class default to the Cayley tranform if no info is provided
        pass
    
    # define the getters
    
    # for the tranforms themselves, remember that z=np.inf or z=cm.infj will likely need to be handled explicitly
    def __call__(self, z):
        pass
    
    def inv(self, z):
        pass
    
    # also include a few factory methods which give other ways to define a Mobius transform:
    @classmethod
    def from3PointMapping(cls, z1to3, w1to3): 
        # use cls since it is the class that is passed instead of the object
        
        # use the implicit form of a MT to calculate the parameters and create
        # an object to return to the user:
        alpha = beta = gamma = delta = 0
        return cls(alpha, beta, gamma, delta)
    
# illustrate the capabilities of your class with test code
        

In [ ]:
# exercises: have them write a class for Mobius transforms
    # it should default to a Cayley transform
    
# closure for translations? what was the concentric circles to confocal ellipses one--Junkowski Tranform